In [4]:
import sys
sys.path.append(".")
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.config import ACTIVITIES, ABBR, TZ_PARIS, TZ_LONDON
from src.viz_style import apply_nature_style

apply_nature_style()

In [6]:
from src.utils_time import to_local_time_series, split_cross_midnight, week_start_monday
from src.utils_split import split_users_by_hash
from src.regularity import regularity_report, summarize_reg, compute_user_hex_stats, infer_home_work_anchors, make_hex_lookup

In [7]:
from pathlib import Path
import joblib

ROOT = Path(".")
OUT_DATA = ROOT / "outputs" / "data"
OUT_MODELS = ROOT / "outputs" / "models"
OUT_FIG = ROOT / "outputs" / "figures"
OUT_FIG.mkdir(parents=True, exist_ok=True)

# load stays
train = pd.read_parquet(OUT_DATA / "paris_stays_train.parquet")
valid = pd.read_parquet(OUT_DATA / "paris_stays_valid.parquet")
for d in [train, valid]:
    d["user_id"] = d["user_id"].astype(str)
    d["start_time"] = pd.to_datetime(d["start_time"])
    d["end_time"] = pd.to_datetime(d["end_time"])
    d["duration_min"] = pd.to_numeric(d["duration_min"], errors="coerce")
    d["hex_id"] = d["hex_id"].astype(str).replace({"": np.nan, "nan": np.nan})
    d["y_true"] = d["y_true"].astype(str)
    d.dropna(subset=["hex_id","start_time","end_time","duration_min","y_true"], inplace=True)

# load Huff POI
poi = pd.read_parquet(OUT_DATA / "paris_poi_huff_k4_b1.5.parquet")
poi["hex_id"] = poi["hex_id"].astype(str)

POI_COLS = ["poi_edu_cnt","poi_health_cnt","poi_retail_cnt","poi_leisure_cnt",
            "poi_transport_cnt","poi_accom_cnt","poi_office_cnt","poi_total_cnt"]
for c in POI_COLS:
    if c not in poi.columns:
        poi[c] = 0

# load GBDT
gbdt = joblib.load(OUT_MODELS / "paris_gbdt_baseline.joblib")
print("Loaded model:", type(gbdt))

# --- rebuild features (copy from 06.5) ---
import h3, math

def cell_to_latlon(cell):
    if hasattr(h3, "cell_to_latlng"):
        lat, lon = h3.cell_to_latlng(cell)
    else:
        lat, lon = h3.h3_to_geo(cell)
    return float(lat), float(lon)

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2-lat1)
    dlmb = math.radians(lon2-lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2*R*math.asin(math.sqrt(a))

all_hex = pd.Index(pd.concat([train["hex_id"], valid["hex_id"]]).unique()).astype(str)
centroids = {h: cell_to_latlon(h) for h in all_hex}

def infer_anchors_and_home_area(stays_df, k_home_area=3):
    d = stays_df.copy().sort_values(["user_id","start_time"])
    d["date"] = d["start_time"].dt.date
    mid = d["start_time"] + pd.to_timedelta(d["duration_min"]/2, unit="m")
    hh = mid.dt.hour + mid.dt.minute/60.0
    night = (hh >= 20) | (hh < 6)
    weekday = (d["start_time"].dt.weekday < 5)
    workhour = weekday & (hh >= 9) & (hh < 17)
    d["night_dwell"] = np.where(night, d["duration_min"], 0.0)
    d["work_dwell"]  = np.where(workhour, d["duration_min"], 0.0)

    g = d.groupby(["user_id","hex_id"], as_index=False).agg(
        night_dwell=("night_dwell","sum"),
        work_dwell=("work_dwell","sum"),
        visit_days=("date", lambda x: x.nunique()),
        dwell=("duration_min","sum"),
    )

    home_hex = g.sort_values(["user_id","night_dwell"], ascending=[True,False]) \
                .drop_duplicates("user_id")[["user_id","hex_id"]].rename(columns={"hex_id":"home_hex"})
    work_hex = g.merge(home_hex, on="user_id", how="left")
    work_hex = work_hex[work_hex["hex_id"] != work_hex["home_hex"]]
    work_hex = work_hex.sort_values(["user_id","work_dwell"], ascending=[True,False]) \
                       .drop_duplicates("user_id")[["user_id","hex_id"]].rename(columns={"hex_id":"work_hex"})

    home_area = g[g["night_dwell"] > 0].sort_values(["user_id","night_dwell"], ascending=[True,False]) \
                  .groupby("user_id").head(k_home_area)
    home_area_lookup = home_area.groupby("user_id")["hex_id"].apply(lambda x: set(x.astype(str))).to_dict()

    anchors = home_hex.merge(work_hex, on="user_id", how="left")
    anchors["home_hex"] = anchors["home_hex"].astype(str)
    anchors["work_hex"] = anchors["work_hex"].astype(str)
    return anchors, home_area_lookup

anchors_train, home_area_train = infer_anchors_and_home_area(train, k_home_area=3)
anchors_valid, home_area_valid = infer_anchors_and_home_area(valid, k_home_area=3)
home_valid = dict(zip(anchors_valid["user_id"], anchors_valid["home_hex"]))
work_valid = dict(zip(anchors_valid["user_id"], anchors_valid["work_hex"]))

def add_poi_features(df, poi_df):
    d = df.merge(poi_df[["hex_id"]+POI_COLS], on="hex_id", how="left").fillna(0)
    sem = d["poi_edu_cnt"] + d["poi_health_cnt"] + d["poi_retail_cnt"] + d["poi_leisure_cnt"]
    d["sem_total"] = sem
    d["edu_ratio"] = d["poi_edu_cnt"]/(sem+1.0)
    d["health_ratio"] = d["poi_health_cnt"]/(sem+1.0)
    d["retail_ratio"] = d["poi_retail_cnt"]/(sem+1.0)
    d["leisure_ratio"] = d["poi_leisure_cnt"]/(sem+1.0)
    return d

def add_time_duration_features(df):
    d = df.copy()
    st = d["start_time"]
    d["hour"] = st.dt.hour
    d["dow"] = st.dt.weekday
    d["is_weekday"] = (d["dow"] < 5).astype(int)
    d["log_dur"] = np.log1p(d["duration_min"].astype(float))
    return d

def add_distance_features(df, home_lookup, work_lookup, home_area_lookup):
    d = df.copy()
    latlon = d["hex_id"].map(lambda h: centroids.get(str(h), (np.nan, np.nan)))
    d["lat"] = [x[0] for x in latlon]
    d["lon"] = [x[1] for x in latlon]

    def anchor_latlon(u, mp):
        hh = mp.get(u, None)
        return centroids.get(str(hh), (np.nan, np.nan))

    home_ll = d["user_id"].map(lambda u: anchor_latlon(u, home_lookup))
    work_ll = d["user_id"].map(lambda u: anchor_latlon(u, work_lookup))
    d["home_lat"] = [x[0] for x in home_ll]; d["home_lon"] = [x[1] for x in home_ll]
    d["work_lat"] = [x[0] for x in work_ll]; d["work_lon"] = [x[1] for x in work_ll]

    d["dist_to_home_km"] = [
        haversine_km(a,b,c,e) if np.isfinite(a) and np.isfinite(c) else np.nan
        for a,b,c,e in zip(d["lat"], d["lon"], d["home_lat"], d["home_lon"])
    ]
    d["dist_to_work_km"] = [
        haversine_km(a,b,c,e) if np.isfinite(a) and np.isfinite(c) else np.nan
        for a,b,c,e in zip(d["lat"], d["lon"], d["work_lat"], d["work_lon"])
    ]

    def dist_to_home_area_min(u, lat, lon):
        hs = home_area_lookup.get(u, None)
        if not hs:
            return np.nan
        best = np.inf
        for hh in hs:
            ll = centroids.get(str(hh))
            if ll is None: 
                continue
            best = min(best, haversine_km(lat, lon, ll[0], ll[1]))
        return best if np.isfinite(best) else np.nan

    d["dist_to_home_area_min_km"] = [
        dist_to_home_area_min(u, la, lo) if np.isfinite(la) else np.nan
        for u, la, lo in zip(d["user_id"], d["lat"], d["lon"])
    ]

    d["near_home_0p5km"] = (d["dist_to_home_km"] <= 0.5).astype(int)
    d["near_work_0p5km"] = (d["dist_to_work_km"] <= 0.5).astype(int)
    return d

def add_prev_distance(df):
    d = df.sort_values(["user_id","start_time"]).copy()
    prev_hex = d.groupby("user_id")["hex_id"].shift(1)
    prev_ll = prev_hex.map(lambda h: centroids.get(str(h), (np.nan, np.nan)))
    prev_lat = [x[0] for x in prev_ll]; prev_lon = [x[1] for x in prev_ll]
    d["dist_prev_km"] = [
        haversine_km(la, lo, pla, plo) if np.isfinite(la) and np.isfinite(pla) else np.nan
        for la,lo,pla,plo in zip(d["lat"], d["lon"], prev_lat, prev_lon)
    ]
    return d

FEATURES = [
    "hour","dow","is_weekday","duration_min","log_dur",
    "poi_edu_cnt","poi_health_cnt","poi_retail_cnt","poi_leisure_cnt","poi_total_cnt",
    "edu_ratio","health_ratio","retail_ratio","leisure_ratio",
    "dist_to_home_km","dist_to_work_km","dist_to_home_area_min_km","near_home_0p5km","near_work_0p5km",
    "dist_prev_km",
]

def build_features(stays):
    d = stays.copy()
    d = add_time_duration_features(d)
    d = add_poi_features(d, poi)
    d = add_distance_features(d, home_valid, work_valid, home_area_valid)
    d = add_prev_distance(d)
    return d

X_valid_df = build_features(valid)
Xva = X_valid_df[FEATURES].replace([np.inf,-np.inf], np.nan)
yva = X_valid_df["y_true"].astype(str).values

Loaded model: <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'>


In [8]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

pred_gbdt = gbdt.predict(Xva)
acc = accuracy_score(yva, pred_gbdt)
macro = f1_score(yva, pred_gbdt, average="macro", labels=ACTIVITIES)
macro_weak = f1_score(yva, pred_gbdt, average="macro", labels=["WORK","STUDY","PURCHASE","LEISURE","HEALTH","OTHER"])

print("GBDT ACC:", acc, "Macro:", macro, "Macro excl HOME:", macro_weak)
print(classification_report(yva, pred_gbdt, labels=ACTIVITIES, digits=3, zero_division=0))

GBDT ACC: 0.6265980541931334 Macro: 0.4907338508592717 Macro excl HOME: 0.4275618679574135
              precision    recall  f1-score   support

        HOME      0.877     0.863     0.870      4153
        WORK      0.799     0.646     0.714      2566
       STUDY      0.158     0.392     0.225       237
    PURCHASE      0.518     0.583     0.548      1832
     LEISURE      0.464     0.512     0.487      1290
      HEALTH      0.148     0.528     0.231       301
       OTHER      0.508     0.278     0.360      2058

    accuracy                          0.627     12437
   macro avg      0.496     0.543     0.491     12437
weighted avg      0.673     0.627     0.638     12437



In [46]:
# reuse TIME_BINS from notebook 06; if not, define quickly
TIME_BINS = [(0,360),(360,600),(600,960),(960,1200),(1200,1440)]

def time_bin_id(minute):
    for i,(a,b) in enumerate(TIME_BINS):
        if a <= minute < b: return i
    return -1

act2i = {a:i for i,a in enumerate(ACTIVITIES)}
i2act = {i:a for a,i in act2i.items()}
K = len(ACTIVITIES)

def split_by_gap(df_u, break_gap_hours=8):
    thr = break_gap_hours * 3600.0
    g = df_u.sort_values("start_time")
    if len(g)==0: return []
    idxs = list(g.index)
    parts=[]; cur=[idxs[0]]
    for a,b in zip(idxs[:-1], idxs[1:]):
        gap = (g.loc[b,"start_time"] - g.loc[a,"end_time"]).total_seconds()
        if gap > thr:
            parts.append(g.loc[cur].copy()); cur=[b]
        else:
            cur.append(b)
    parts.append(g.loc[cur].copy())
    return parts

def build_A_dict_from_stays(stays_df, alpha=1.0, rho=0.3, break_gap_hours=8):
    df = stays_df.sort_values(["user_id","start_time"]).copy()

    def ctx(dt):
        w = int(dt.weekday() >= 5)
        tb = time_bin_id(int(dt.hour*60 + dt.minute))
        return (w, tb)

    C_global = np.zeros((K,K), float)
    C_bucket = {}

    for u, g in df.groupby("user_id", sort=False):
        parts = split_by_gap(g, break_gap_hours=break_gap_hours)
        for seq in parts:
            y = seq["y_true"].astype(str).values
            for i in range(len(seq)-1):
                a, b = y[i], y[i+1]
                if a not in act2i or b not in act2i:
                    continue
                dt = seq.iloc[i]["start_time"]
                w, tb = ctx(dt)
                if tb < 0:
                    continue
                if (w,tb) not in C_bucket:
                    C_bucket[(w,tb)] = np.zeros((K,K), float)
                ii, jj = act2i[a], act2i[b]
                C_bucket[(w,tb)][ii, jj] += 1.0
                C_global[ii, jj] += 1.0

    def row_norm(C):
        A = C + alpha
        A = A / A.sum(axis=1, keepdims=True)
        return A

    A_global = row_norm(C_global)
    A_dict = {}
    for w in [0,1]:
        for tb in range(len(TIME_BINS)):
            C = C_bucket.get((w,tb))
            if C is None:
                A_dict[(w,tb)] = A_global.copy()
            else:
                A = row_norm(C)
                A = (1-rho)*A + rho*A_global
                A = A / A.sum(axis=1, keepdims=True)
                A_dict[(w,tb)] = A
    return A_dict

A_dict = build_A_dict_from_stays(train, alpha=1.0, rho=0.3, break_gap_hours=8)

def A_getter(seq, t):
    dt = seq.iloc[t]["start_time"]
    w = int(dt.weekday() >= 5)
    tb = time_bin_id(int(dt.hour*60 + dt.minute))
    return A_dict[(w,tb)]

In [47]:
# # Build TRAIN features (same pipeline as valid)
# X_train_df = build_features(train)  # build_features() you defined in Cell 1
# Xtr = X_train_df[FEATURES].replace([np.inf, -np.inf], np.nan)
# ytr = X_train_df["y_true"].astype(str).values

# print("Xtr shape:", Xtr.shape, "train rows:", len(X_train_df), "train users:", X_train_df["user_id"].nunique())

# proba_tr = gbdt.predict_proba(Xtr)   # Xtr 是 train 的 features
# classes = list(gbdt.classes_)
# idx_map = [classes.index(a) for a in ACTIVITIES]
# proba_tr = proba_tr[:, idx_map]      # align to ACTIVITIES order


# train_feat = X_train_df.sort_values(["user_id","start_time"]).copy()
# train_feat["proba_row"] = list(proba_tr)


# def build_A_dict_from_proba(train_feat, alpha=1.0, rho=0.3, break_gap_hours=8):
#     # train_feat: sorted by user_id,start_time, has start_time,end_time, proba_row (len K)
#     C_global = np.zeros((K,K), float)
#     C_bucket = {}

#     def ctx(dt):
#         w = int(dt.weekday() >= 5)
#         tb = time_bin_id(int(dt.hour*60 + dt.minute))
#         return (w, tb)

#     for u, g in train_feat.groupby("user_id", sort=False):
#         parts = split_by_gap(g, break_gap_hours=break_gap_hours)
#         for seq in parts:
#             for t in range(len(seq)-1):
#                 dt = seq.iloc[t]["start_time"]
#                 w, tb = ctx(dt)
#                 if tb < 0: 
#                     continue
#                 if (w,tb) not in C_bucket:
#                     C_bucket[(w,tb)] = np.zeros((K,K), float)

#                 p = np.array(seq.iloc[t]["proba_row"], dtype=float)
#                 q = np.array(seq.iloc[t+1]["proba_row"], dtype=float)
#                 C = np.outer(p, q)           # expected transition counts
#                 C_bucket[(w,tb)] += C
#                 C_global += C

#     def row_norm(C):
#         A = C + alpha
#         A = A / A.sum(axis=1, keepdims=True)
#         return A

#     A_global = row_norm(C_global)
#     A_dict = {}
#     for w in [0,1]:
#         for tb in range(len(TIME_BINS)):
#             C = C_bucket.get((w,tb))
#             if C is None:
#                 A_dict[(w,tb)] = A_global.copy()
#             else:
#                 A = row_norm(C)
#                 A = (1-rho)*A + rho*A_global
#                 A = A / A.sum(axis=1, keepdims=True)
#                 A_dict[(w,tb)] = A
#     return A_dict

# A_dict_soft = build_A_dict_from_proba(train_feat, alpha=1.0, rho=0.3, break_gap_hours=8)

# def A_getter(seq, t):
#     dt = seq.iloc[t]["start_time"]
#     w = int(dt.weekday() >= 5)
#     tb = time_bin_id(int(dt.hour*60 + dt.minute))
#     return A_dict_soft[(w,tb)]

In [48]:
def logsumexp(v):
    m = np.max(v)
    return float(m + np.log(np.sum(np.exp(v - m))))

def viterbi_decode(logB, seq, self_loop_bonus=0.2):
    eps = 1e-12
    T, K0 = logB.shape
    log_pi = np.log(np.full(K0, 1.0/K0))
    dp = np.full((T,K0), -np.inf)
    bp = np.zeros((T,K0), dtype=int)
    dp[0] = log_pi + logB[0]
    bp[0] = -1

    for t in range(1,T):
        A = A_getter(seq, t-1)
        logA = np.log(A + eps)
        for j in range(K0):
            scores = dp[t-1] + logA[:,j] + self_loop_bonus*(np.arange(K0)==j)
            i_star = int(np.argmax(scores))
            bp[t,j] = i_star
            dp[t,j] = scores[i_star] + logB[t,j]

    last = int(np.argmax(dp[T-1]))
    path=[last]
    for t in range(T-1,0,-1):
        last = bp[t,last]
        path.append(last)
    return np.array(path[::-1], dtype=int)

# proba -> logB aligned to ACTIVITIES order
proba = gbdt.predict_proba(Xva)  # shape (N, n_classes_in_model)
classes = list(gbdt.classes_)
idx_map = [classes.index(a) for a in ACTIVITIES]  # assume all present
proba_aligned = proba[:, idx_map]

T=0.5
logB_all = np.log(np.clip(proba_aligned, 1e-12, 1.0)) / T

# decode per user segment
df = X_valid_df.copy().sort_values(["user_id","start_time"])
df["logB_row"] = list(logB_all)  # store row vectors

df["y_pred_hybrid"] = None

for u, g in df.groupby("user_id", sort=False):
    parts = split_by_gap(g, break_gap_hours=8)
    for seq in parts:
        logB = np.stack(seq["logB_row"].values, axis=0)
        path = viterbi_decode(logB, seq, self_loop_bonus=0.1)
        df.loc[seq.index, "y_pred_hybrid"] = [i2act[int(i)] for i in path]

# evaluate
from sklearn.metrics import accuracy_score, f1_score, classification_report

y_pred_hybrid = df["y_pred_hybrid"].astype(str).values
acc_h = accuracy_score(yva, y_pred_hybrid)
macro_h = f1_score(yva, y_pred_hybrid, average="macro", labels=ACTIVITIES)
macro_weak_h = f1_score(yva, y_pred_hybrid, average="macro", labels=["WORK","STUDY","PURCHASE","LEISURE","HEALTH","OTHER"])

print("Hybrid (GBDT emission + HMM) ACC:", acc_h, "Macro:", macro_h, "Macro excl HOME:", macro_weak_h)
print(classification_report(yva, y_pred_hybrid, labels=ACTIVITIES, digits=3, zero_division=0))

Hybrid (GBDT emission + HMM) ACC: 0.6690520221918469 Macro: 0.5128403271320069 Macro excl HOME: 0.45004974257133634
              precision    recall  f1-score   support

        HOME      0.889     0.890     0.890      4153
        WORK      0.752     0.769     0.760      2566
       STUDY      0.224     0.194     0.208       237
    PURCHASE      0.515     0.635     0.569      1832
     LEISURE      0.486     0.511     0.498      1290
      HEALTH      0.219     0.346     0.268       301
       OTHER      0.500     0.329     0.397      2058

    accuracy                          0.669     12437
   macro avg      0.512     0.525     0.513     12437
weighted avg      0.670     0.669     0.665     12437



In [40]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(yva, y_pred_hybrid, labels=ACTIVITIES)
cmn = cm / np.maximum(cm.sum(axis=1, keepdims=True), 1)

cm_csv = OUT_DATA / "cm_hybrid_gbdt_hmm_row_norm.csv"
np.savetxt(cm_csv, cmn, delimiter=",", fmt="%.6f")
print("Saved:", cm_csv)

Saved: /Users/pang/Codes/GISRUK/outputs/data/cm_hybrid_gbdt_hmm_row_norm.csv


In [51]:
def save_row_norm_cm(y_true, y_pred, out_csv):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cmn = cm / np.maximum(cm.sum(axis=1, keepdims=True), 1)
    np.savetxt(out_csv, cmn, delimiter=",", fmt="%.6f")
    print("Saved:", out_csv)
    return cmn

In [52]:
save_row_norm_cm(yva, y_pred_hybrid, OUT_DATA / "cm_hybrid_row_norm.csv")

Saved: /Users/pang/Codes/GISRUK/outputs/data/cm_hybrid_row_norm.csv


array([[0.89044065, 0.02311582, 0.00120395, 0.03106188, 0.02167108,
        0.00722369, 0.02528293],
       [0.03546376, 0.76890101, 0.04988309, 0.05144193, 0.03312549,
        0.020265  , 0.04091972],
       [0.03375527, 0.58649789, 0.19409283, 0.02109705, 0.07594937,
        0.01265823, 0.07594937],
       [0.05131004, 0.04257642, 0.0010917 , 0.63537118, 0.07914847,
        0.06058952, 0.12991266],
       [0.05116279, 0.09379845, 0.01085271, 0.16124031, 0.51085271,
        0.04651163, 0.1255814 ],
       [0.03322259, 0.09966777, 0.        , 0.27242525, 0.08305648,
        0.34551495, 0.16611296],
       [0.09426628, 0.09135083, 0.00485909, 0.26287658, 0.16229349,
        0.05539359, 0.32896016]])

In [53]:
from pathlib import Path
OUT_DATA = Path(".")
OUT_DATA.mkdir(parents=True, exist_ok=True)

df_out = X_valid_df[["user_id","start_time","end_time","hex_id","duration_min","y_true"]].copy()
df_out["y_pred"] = y_pred_hybrid
df_out.to_parquet(OUT_DATA / "paris_valid_pred_hybrid.parquet", index=False)
print("Saved:", OUT_DATA / "paris_valid_pred_hybrid.parquet")

Saved: /Users/pang/Codes/GISRUK/outputs/data/paris_valid_pred_hybrid.parquet
